# <center>Naive Bayes Tutorial Part 1: Predicting survival from titanic crash<center>

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [3]:
df.Survived.value_counts()

Survived
0    549
1    342
Name: count, dtype: int64

In [4]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [5]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived

In [6]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,False,True
1,True,False
2,True,False


In [7]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,False,True
1,1,female,38.0,71.2833,True,False
2,3,female,26.0,7.9250,True,False


I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female

In [8]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True


In [9]:
inputs.columns[inputs.isna().any()]

Index(['Age'], dtype='object')

In [10]:
inputs.Age[:10]

0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [11]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()

,Pclass,Age,Fare,female
0,3,22.0,7.2500,False
1,1,38.0,71.2833,True
2,3,26.0,7.9250,True
3,1,35.0,53.1000,True
4,3,35.0,8.0500,False


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.3)

In [13]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [14]:
model.fit(X_train,y_train)

GaussianNB()

In [15]:
model.score(X_test,y_test)

0.7686567164179104

In [16]:
y_pred = model.predict(X_test)

In [20]:
from sklearn.metrics import accuracy_score
print("Accuracy",accuracy_score(y_pred,y_test))

Accuracy 0.7686567164179104


In [21]:
X_test[0:10]

,Pclass,Age,Fare,female
488,3,30.0,8.0500,False
489,3,9.0,15.9000,False
625,1,61.0,32.3208,False
382,3,32.0,7.9250,False
580,2,25.0,30.0000,True
125,3,12.0,11.2417,False
286,3,30.0,9.5000,False
645,1,48.0,76.7292,False
153,3,40.5,14.5000,False
421,3,21.0,7.7333,False


In [22]:
y_test[0:10]

488    0
489    1
625    0
382    0
580    1
125    1
286    1
645    1
153    0
421    0
Name: Survived, dtype: int64

In [23]:
model.predict_proba(X_test[:10])

array([[0.96942595, 0.03057405],
       [0.95037238, 0.04962762],
       [0.69707844, 0.30292156],
       [0.97000852, 0.02999148],
       [0.21048343, 0.78951657],
       [0.95522415, 0.04477585],
       [0.96954521, 0.03045479],
       [0.45837977, 0.54162023],
       [0.9711164 , 0.0288836 ],
       [0.96455107, 0.03544893]])

Calculate the score using cross validation

In [25]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.76      , 0.736     , 0.776     , 0.78225806, 0.84677419])